<h3> prøver å gjenskape henning sin 143.</h3>
<p><p>

In [1]:
%load_ext autoreload

In [2]:
from sklearn.model_selection import KFold
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np

/Users/miksx/.pyenv/versions/3.11.4/envs/machinelearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%autoreload
from pipeline import Pipeline
pipin = Pipeline()

Current working directory: /Users/miksx/GitHub/Forest-Gump/mikael/optuna


In [4]:
PATH = "ag_144_"

In [5]:
df1_0 = pipin.get_data("A")
df2_0 = pipin.get_data("B")
df3_0 = pipin.get_data("C")

In [6]:
# remove_consecutive_measurments
# 6/24
# dette skal prøve å gjenskape henning sin
df1_0 = pipin.remove_consecutive_measurments_new(df1_0, 4, 12)
df2_0 = pipin.remove_consecutive_measurments_new(df2_0, 4, 12)
df3_0 = pipin.remove_consecutive_measurments_new(df3_0, 4, 12)

In [7]:
df1_0 = pipin.drop_columns(df1_0)
df2_0 = pipin.drop_columns(df2_0)
df3_0 = pipin.drop_columns(df3_0)
print(df1_0.shape, df2_0.shape, df3_0.shape)

(23231, 47) (15838, 47) (13238, 47)


In [8]:
df1_0.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,estimated,observed,day_of_year,hour,pv_measurement
0,7.700,1.22825,1728.949951,0.000000,0.00,1728.949951,0.0,280.299988,0.000,0.000000,...,0.0,286.225006,100.000000,40386.476562,3.600,0.0,1.0,153.0,22.0,0.00
1,7.700,1.22350,1689.824951,0.000000,0.00,1689.824951,0.0,280.299988,0.000,0.000000,...,0.0,286.899994,100.000000,33770.648438,3.350,0.0,1.0,153.0,23.0,0.00
2,7.875,1.21975,1563.224976,0.000000,0.00,1563.224976,0.0,280.649994,0.000,0.000000,...,0.0,286.950012,100.000000,13595.500000,3.050,0.0,1.0,154.0,0.0,0.00
3,8.425,1.21800,1283.425049,208.649994,0.75,1283.425049,0.0,281.674988,0.300,526.775024,...,0.0,286.750000,100.000000,2321.850098,2.725,0.0,1.0,154.0,1.0,0.00
4,8.950,1.21800,1003.500000,32468.150391,23.10,1003.500000,0.0,282.500000,11.975,22068.949219,...,0.0,286.450012,99.224998,11634.799805,2.550,0.0,1.0,154.0,2.0,19.36


In [9]:
def get_objective(X=None, y=None):
    def objective(trial):
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        params = {
            "iterations": 500,
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "depth": trial.suggest_int("depth", 4, 10),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        }
        # Initialize a list to store the mean absolute errors for each fold
        mae_list = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
            X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
            y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]

            print(f"Training Fold {fold}...")
            model = CatBoostRegressor(**params, silent=True)
            model.fit(X_fold_train, y_fold_train)
            predictions = model.predict(X_fold_val)
            mae_fold = mean_absolute_error(y_fold_val, predictions)
            mae_list.append(mae_fold)
            print(f"Fold {fold} MAE: {mae_fold}")

        # Calculate the mean of the mean absolute errors for all folds
        mae = sum(mae_list) / len(mae_list)
        print(f"Mean MAE for all Folds: {mae}")
        return mae
    return objective

In [10]:
X_1, y_1 = df1_0[[c for c in df1_0.columns if "pv_measurement" != c]], df1_0["pv_measurement"]

In [11]:
study1 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study1.optimize(get_objective(X=X_1,y=y_1 ), n_trials=30)
print("Optimization complete!")

predictor1 = CatBoostRegressor(**study1.best_params).fit(X_1, y_1)


[I 2023-11-09 11:38:08,564] A new study created in memory with name: no-name-9f59bcc2-63f1-4d72-8251-198c6fdd158e


Optimizing hyperparameters...
Training Fold 1...
Fold 1 MAE: 555.8332470022201
Training Fold 2...
Fold 2 MAE: 541.0647275107564
Training Fold 3...
Fold 3 MAE: 542.7807840677115
Training Fold 4...
Fold 4 MAE: 551.2757468631812
Training Fold 5...
Fold 5 MAE: 561.3570382595282
Training Fold 6...
Fold 6 MAE: 544.9810083104182
Training Fold 7...
Fold 7 MAE: 547.9182655768556
Training Fold 8...
Fold 8 MAE: 542.2711963275951
Training Fold 9...
Fold 9 MAE: 571.3487692752485
Training Fold 10...


[I 2023-11-09 11:38:37,515] Trial 0 finished with value: 551.4198271120747 and parameters: {'learning_rate': 0.001849429189477468, 'depth': 10, 'subsample': 0.2839178582585555, 'colsample_bylevel': 0.388985528740475, 'min_data_in_leaf': 64}. Best is trial 0 with value: 551.4198271120747.


Fold 10 MAE: 555.367487927232
Mean MAE for all Folds: 551.4198271120747
Training Fold 1...
Fold 1 MAE: 414.6956640893128
Training Fold 2...
Fold 2 MAE: 414.3029053333076
Training Fold 3...
Fold 3 MAE: 409.00727191600464
Training Fold 4...
Fold 4 MAE: 412.30228581673487
Training Fold 5...
Fold 5 MAE: 423.78240202735805
Training Fold 6...
Fold 6 MAE: 411.72323429933397
Training Fold 7...
Fold 7 MAE: 413.7553485666827
Training Fold 8...
Fold 8 MAE: 407.2200571384664
Training Fold 9...
Fold 9 MAE: 430.88633674982566
Training Fold 10...


[I 2023-11-09 11:38:49,089] Trial 1 finished with value: 415.42531798188804 and parameters: {'learning_rate': 0.003391387468846783, 'depth': 6, 'subsample': 0.376135372812308, 'colsample_bylevel': 0.7459734475239754, 'min_data_in_leaf': 41}. Best is trial 1 with value: 415.42531798188804.


...

Fold 10 MAE: 245.1259955245667
Mean MAE for all Folds: 245.07514560321334
Training Fold 1...
Fold 1 MAE: 238.60073468268962
Training Fold 2...
Fold 2 MAE: 253.69820539533228
Training Fold 3...
Fold 3 MAE: 238.97194936508657
Training Fold 4...
Fold 4 MAE: 240.54255411247942
Training Fold 5...
Fold 5 MAE: 246.20839698173688
Training Fold 6...
Fold 6 MAE: 245.8724795512624
Training Fold 7...
Fold 7 MAE: 246.88364493895264
Training Fold 8...
Fold 8 MAE: 233.54920038977608
Training Fold 9...
Fold 9 MAE: 251.37054920017394
Training Fold 10...


[I 2023-11-09 11:56:29,534] Trial 28 finished with value: 243.96714502906963 and parameters: {'learning_rate': 0.09812670932735298, 'depth': 10, 'subsample': 0.9568488594744029, 'colsample_bylevel': 0.8746135787435793, 'min_data_in_leaf': 72}. Best is trial 28 with value: 243.96714502906963.


Fold 10 MAE: 243.97373567320633
Mean MAE for all Folds: 243.96714502906963
Training Fold 1...
Fold 1 MAE: 240.06519667769686
Training Fold 2...
Fold 2 MAE: 255.42537601161908
Training Fold 3...
Fold 3 MAE: 241.02523908772875
Training Fold 4...
Fold 4 MAE: 240.4662306231402
Training Fold 5...
Fold 5 MAE: 248.9854463555152
Training Fold 6...
Fold 6 MAE: 244.03638679920897
Training Fold 7...
Fold 7 MAE: 245.46863414510548
Training Fold 8...
Fold 8 MAE: 237.64361315222448
Training Fold 9...
Fold 9 MAE: 253.68711104148235
Training Fold 10...


[I 2023-11-09 11:59:21,034] Trial 29 finished with value: 245.1670450412874 and parameters: {'learning_rate': 0.0962056193098205, 'depth': 10, 'subsample': 0.9981994893734037, 'colsample_bylevel': 0.8501580823990704, 'min_data_in_leaf': 64}. Best is trial 28 with value: 243.96714502906963.


Fold 10 MAE: 244.86721651915292
Mean MAE for all Folds: 245.1670450412874
Optimization complete!
0:	learn: 1215.4397001	total: 36.7ms	remaining: 36.6s
1:	learn: 1127.3643883	total: 67.6ms	remaining: 33.7s
2:	learn: 1049.7752636	total: 144ms	remaining: 47.9s
3:	learn: 979.9147222	total: 187ms	remaining: 46.4s
4:	learn: 918.2284204	total: 229ms	remaining: 45.6s
5:	learn: 863.7883409	total: 265ms	remaining: 43.9s
6:	learn: 817.8738275	total: 312ms	remaining: 44.3s
7:	learn: 775.8273873	total: 353ms	remaining: 43.7s
8:	learn: 740.1699941	total: 398ms	remaining: 43.8s
9:	learn: 709.0211403	total: 448ms	remaining: 44.4s
10:	learn: 682.2451899	total: 501ms	remaining: 45.1s
11:	learn: 657.9794900	total: 551ms	remaining: 45.4s
12:	learn: 637.4862317	total: 602ms	remaining: 45.7s
13:	learn: 618.4475201	total: 678ms	remaining: 47.7s
14:	learn: 602.8075455	total: 715ms	remaining: 46.9s
15:	learn: 589.8082669	total: 748ms	remaining: 46s
16:	learn: 576.6470006	total: 774ms	remaining: 44.8s
17:	learn

In [12]:
X_2, y_2 = df2_0[[c for c in df2_0.columns if "pv_measurement" != c]], df2_0["pv_measurement"]

In [13]:
study2 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study2.optimize(get_objective(X=X_2,y=y_2 ), n_trials=30)
print("Optimization complete!")
predictor2 = CatBoostRegressor(**study2.best_params).fit(X_2, y_2)


[I 2023-11-09 12:00:11,885] A new study created in memory with name: no-name-b6e8c0bc-ebb4-4fa1-a3e3-ce3fda2e5527


Optimizing hyperparameters...
Training Fold 1...
Fold 1 MAE: 44.165838609555045
Training Fold 2...
Fold 2 MAE: 38.42505243724871
Training Fold 3...
Fold 3 MAE: 39.77783042125512
Training Fold 4...
Fold 4 MAE: 41.13404103899804
Training Fold 5...
Fold 5 MAE: 41.612389494113394
Training Fold 6...
Fold 6 MAE: 41.20770324741108
Training Fold 7...
Fold 7 MAE: 41.901150740015034
Training Fold 8...
Fold 8 MAE: 40.18352168837885
Training Fold 9...
Fold 9 MAE: 43.6188824900861
Training Fold 10...


[I 2023-11-09 12:01:32,205] Trial 0 finished with value: 41.44634859674106 and parameters: {'learning_rate': 0.05737060294494729, 'depth': 8, 'subsample': 0.7180641966970279, 'colsample_bylevel': 0.564687265160978, 'min_data_in_leaf': 1}. Best is trial 0 with value: 41.44634859674106.


Fold 10 MAE: 42.43707580034926
Mean MAE for all Folds: 41.44634859674106
Training Fold 1...
Fold 1 MAE: 132.76754451790364
Training Fold 2...
Fold 2 MAE: 130.71981273556216
Training Fold 3...
Fold 3 MAE: 131.73410406515933
Training Fold 4...
Fold 4 MAE: 134.70638061349578
Training Fold 5...
Fold 5 MAE: 132.0169622244916
Training Fold 6...
Fold 6 MAE: 132.06084017319154
Training Fold 7...
Fold 7 MAE: 136.05958520258008
Training Fold 8...
Fold 8 MAE: 133.19389808808305
Training Fold 9...
Fold 9 MAE: 133.2729545751434
Training Fold 10...


[I 2023-11-09 12:01:46,326] Trial 1 finished with value: 133.0355872461169 and parameters: {'learning_rate': 0.0010260135059696593, 'depth': 4, 'subsample': 0.5307927979689968, 'colsample_bylevel': 0.1946462811842663, 'min_data_in_leaf': 35}. Best is trial 0 with value: 41.44634859674106.


...

Fold 10 MAE: 41.45529539398825
Mean MAE for all Folds: 40.181590438641706
Training Fold 1...
Fold 1 MAE: 42.88603793256033
Training Fold 2...
Fold 2 MAE: 37.65140562635281
Training Fold 3...
Fold 3 MAE: 38.45131294810398
Training Fold 4...
Fold 4 MAE: 39.18727375764388
Training Fold 5...
Fold 5 MAE: 40.18438146474534
Training Fold 6...
Fold 6 MAE: 40.28268344261892
Training Fold 7...
Fold 7 MAE: 40.65669358968805
Training Fold 8...
Fold 8 MAE: 38.880304803850535
Training Fold 9...
Fold 9 MAE: 41.86274946751644
Training Fold 10...


[I 2023-11-09 12:18:05,054] Trial 28 finished with value: 40.20087930918827 and parameters: {'learning_rate': 0.04825023936605068, 'depth': 10, 'subsample': 0.6595905441484791, 'colsample_bylevel': 0.3368339551255536, 'min_data_in_leaf': 57}. Best is trial 22 with value: 38.921313668423565.


Fold 10 MAE: 41.96595005880239
Mean MAE for all Folds: 40.20087930918827
Training Fold 1...
Fold 1 MAE: 44.03502206710159
Training Fold 2...
Fold 2 MAE: 38.59528378089937
Training Fold 3...
Fold 3 MAE: 39.553197037590664
Training Fold 4...
Fold 4 MAE: 40.386831770509026
Training Fold 5...
Fold 5 MAE: 41.59293863204223
Training Fold 6...
Fold 6 MAE: 41.36902167514964
Training Fold 7...
Fold 7 MAE: 41.9813396736846
Training Fold 8...
Fold 8 MAE: 39.780113194174795
Training Fold 9...
Fold 9 MAE: 42.79572636802245
Training Fold 10...


[I 2023-11-09 12:18:37,581] Trial 29 finished with value: 41.24913438148537 and parameters: {'learning_rate': 0.06216764267475136, 'depth': 8, 'subsample': 0.8788364064428961, 'colsample_bylevel': 0.5659704241239798, 'min_data_in_leaf': 2}. Best is trial 22 with value: 38.921313668423565.


Fold 10 MAE: 42.40186961567927
Mean MAE for all Folds: 41.24913438148537
Optimization complete!
0:	learn: 229.7254035	total: 8.19ms	remaining: 8.18s
1:	learn: 212.2758739	total: 15.6ms	remaining: 7.79s
2:	learn: 196.5632925	total: 23.3ms	remaining: 7.73s
3:	learn: 182.3902947	total: 31.1ms	remaining: 7.74s
4:	learn: 169.9221899	total: 37.2ms	remaining: 7.39s
5:	learn: 158.8760579	total: 43.6ms	remaining: 7.22s
6:	learn: 149.1972977	total: 50.3ms	remaining: 7.14s
7:	learn: 140.6265704	total: 58.8ms	remaining: 7.29s
8:	learn: 133.2251347	total: 66.1ms	remaining: 7.28s
9:	learn: 126.5032246	total: 74.3ms	remaining: 7.35s
10:	learn: 120.7427924	total: 83.2ms	remaining: 7.48s
11:	learn: 115.7610113	total: 89.6ms	remaining: 7.38s
12:	learn: 111.3840419	total: 96.8ms	remaining: 7.35s
13:	learn: 107.7247625	total: 104ms	remaining: 7.34s
14:	learn: 104.4763586	total: 111ms	remaining: 7.3s
15:	learn: 101.6310589	total: 118ms	remaining: 7.26s
16:	learn: 99.0780770	total: 125ms	remaining: 7.25s
17

In [14]:
X_3, y_3 = df3_0[[c for c in df3_0.columns if "pv_measurement" != c]], df3_0["pv_measurement"]

In [15]:
study3 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study3.optimize(get_objective(X=X_3,y=y_3 ), n_trials=30)
print("Optimization complete!")
predictor3 = CatBoostRegressor(**study3.best_params).fit(X_3, y_3)


[I 2023-11-09 12:18:45,141] A new study created in memory with name: no-name-35bef250-a2ce-49f4-bb4b-9fddca18acec


Optimizing hyperparameters...
Training Fold 1...
Fold 1 MAE: 106.03825373202493
Training Fold 2...
Fold 2 MAE: 105.83299213016636
Training Fold 3...
Fold 3 MAE: 106.50108247766032
Training Fold 4...
Fold 4 MAE: 104.21167265770276
Training Fold 5...
Fold 5 MAE: 109.86837016648464
Training Fold 6...
Fold 6 MAE: 109.35629669935037
Training Fold 7...
Fold 7 MAE: 109.13030046498255
Training Fold 8...
Fold 8 MAE: 109.48453771676449
Training Fold 9...
Fold 9 MAE: 112.06238735167108
Training Fold 10...


[I 2023-11-09 12:19:05,685] Trial 0 finished with value: 108.11615914857566 and parameters: {'learning_rate': 0.001002577497528038, 'depth': 7, 'subsample': 0.4813189036933892, 'colsample_bylevel': 0.6331593654339541, 'min_data_in_leaf': 26}. Best is trial 0 with value: 108.11615914857566.


Fold 10 MAE: 108.675698088949
Mean MAE for all Folds: 108.11615914857566
Training Fold 1...
Fold 1 MAE: 40.32425987385058
Training Fold 2...
Fold 2 MAE: 37.95691887298799
Training Fold 3...
Fold 3 MAE: 40.37938826002487
Training Fold 4...
Fold 4 MAE: 37.03213079824062
Training Fold 5...
Fold 5 MAE: 40.28775655232272
Training Fold 6...
Fold 6 MAE: 41.42469147639382
Training Fold 7...
Fold 7 MAE: 40.48411569538389
Training Fold 8...
Fold 8 MAE: 39.57083222849053
Training Fold 9...
Fold 9 MAE: 40.30229602759044
Training Fold 10...


[I 2023-11-09 12:19:24,310] Trial 1 finished with value: 39.71664659188124 and parameters: {'learning_rate': 0.016099718501177383, 'depth': 7, 'subsample': 0.09039680894542426, 'colsample_bylevel': 0.7029754241059561, 'min_data_in_leaf': 38}. Best is trial 1 with value: 39.71664659188124.


...

Fold 10 MAE: 35.370859777905984
Mean MAE for all Folds: 35.51562561621615
Training Fold 1...
Fold 1 MAE: 34.49408415970037
Training Fold 2...
Fold 2 MAE: 32.71858221200998
Training Fold 3...
Fold 3 MAE: 34.35439265496888
Training Fold 4...
Fold 4 MAE: 31.88742544728156
Training Fold 5...
Fold 5 MAE: 34.02240494847843
Training Fold 6...
Fold 6 MAE: 35.83458100552792
Training Fold 7...
Fold 7 MAE: 34.51947816111001
Training Fold 8...
Fold 8 MAE: 33.90760208249741
Training Fold 9...
Fold 9 MAE: 33.77519645443173
Training Fold 10...


[I 2023-11-09 12:36:32,757] Trial 28 finished with value: 34.00638012347741 and parameters: {'learning_rate': 0.09837597976257814, 'depth': 9, 'subsample': 0.8507686946203219, 'colsample_bylevel': 0.9222819367005268, 'min_data_in_leaf': 71}. Best is trial 21 with value: 33.47285593276349.


Fold 10 MAE: 34.55005410876781
Mean MAE for all Folds: 34.00638012347741
Training Fold 1...
Fold 1 MAE: 36.198978259677155
Training Fold 2...
Fold 2 MAE: 34.43789766090979
Training Fold 3...
Fold 3 MAE: 36.80893492288597
Training Fold 4...
Fold 4 MAE: 33.88982229097196
Training Fold 5...
Fold 5 MAE: 36.61095818569429
Training Fold 6...
Fold 6 MAE: 37.42384414317168
Training Fold 7...
Fold 7 MAE: 35.913571926617024
Training Fold 8...
Fold 8 MAE: 35.818465963840076
Training Fold 9...
Fold 9 MAE: 35.632078311200154
Training Fold 10...


[I 2023-11-09 12:36:52,595] Trial 29 finished with value: 35.90543202823763 and parameters: {'learning_rate': 0.06628685969236817, 'depth': 7, 'subsample': 0.5277721399798816, 'colsample_bylevel': 0.6073378224701412, 'min_data_in_leaf': 58}. Best is trial 21 with value: 33.47285593276349.


Fold 10 MAE: 36.31976861740823
Mean MAE for all Folds: 35.90543202823763
Optimization complete!
0:	learn: 191.6522679	total: 11ms	remaining: 11s
1:	learn: 178.1186638	total: 21.4ms	remaining: 10.7s
2:	learn: 166.5306864	total: 32.3ms	remaining: 10.7s
3:	learn: 155.3861829	total: 42.2ms	remaining: 10.5s
4:	learn: 145.6539895	total: 52ms	remaining: 10.3s
5:	learn: 136.9505100	total: 62.2ms	remaining: 10.3s
6:	learn: 129.0234523	total: 73.6ms	remaining: 10.4s
7:	learn: 122.0386385	total: 83.5ms	remaining: 10.4s
8:	learn: 115.8452194	total: 94.9ms	remaining: 10.5s
9:	learn: 110.2100813	total: 105ms	remaining: 10.4s
10:	learn: 105.2974340	total: 117ms	remaining: 10.5s
11:	learn: 100.9931610	total: 129ms	remaining: 10.6s
12:	learn: 97.1657247	total: 140ms	remaining: 10.6s
13:	learn: 93.7568046	total: 150ms	remaining: 10.6s
14:	learn: 90.7222398	total: 161ms	remaining: 10.6s
15:	learn: 87.9617280	total: 172ms	remaining: 10.6s
16:	learn: 85.6401704	total: 181ms	remaining: 10.5s
17:	learn: 83.5

In [16]:
test1 = pipin.get_test_data("A")
test2 = pipin.get_test_data("B")
test3 = pipin.get_test_data("C")

test1 = pipin.drop_columns(test1)
test2 = pipin.drop_columns(test2)
test3 = pipin.drop_columns(test3)

In [17]:
pred1 = pd.DataFrame(predictor1.predict(test1), columns=["pv_measurement"])
pred2 = pd.DataFrame(predictor2.predict(test2), columns=["pv_measurement"])
pred3 = pd.DataFrame(predictor3.predict(test3), columns=["pv_measurement"])
negatives_pred1 = pred1[pred1["pv_measurement"] < 0]
negatives_pred2 = pred2[pred2["pv_measurement"] < 0]
negatives_pred3 = pred3[pred3["pv_measurement"] < 0]
neg = pd.concat([negatives_pred1, negatives_pred2, negatives_pred3])
neg.shape

(189, 1)

In [18]:
pred = pd.concat([pred1, pred2, pred3])
final_prediction = pipin.post_processing(pred, prediction_column="pv_measurement")
final_prediction.to_csv('submissions/catboost_143_pipeline.csv')

In [19]:
diff = pipin.compare_mae(final_prediction)
diff

54.58110570779545

In [20]:
# predictor.evaluate(df1, silent=True)